In [12]:
import pandas as pd
import os
from options import *
import json
import sys
sys.argv = ['test.py']



args = get_args()

with open(args.best_model, 'r+') as f:
    best_model = json.load(f)
    loss, MACC = 20, 99
    if best_model['loss'] == None or loss < best_model['loss'] :
        best_model['name'] = "CVPR"
        best_model['loss'] = loss
        best_model['MACC'] = MACC
        best_model['path'] = os.path.join(args.best_model_save_dir , "best_{}.pth".format(args.name))
    f.seek(0)
    json.dump(best_model, f, indent=4, ensure_ascii=False)
f.close()